## pandas&csv

#### get data

In [68]:
import pandas as pd

- open csv file

In [69]:
path = 'data/melb_data.csv'
data = pd.read_csv(path)
#column count
data.describe() # data: DataFrame

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


- list of columns in dataset

In [70]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

- dropna drops missing values (think of na as "not available")

In [71]:
data = data.dropna(axis=0)

- we use dot-notation to select the column we want to predict

In [72]:
y = data.Price

- the columns are inputted into model are called features

In [73]:
features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = data[features]
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


- top few rows

In [74]:
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


#### use scikit-learn to build model
- decision tree

In [75]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
model = DecisionTreeRegressor()
# Fit model
model.fit(X, y)

DecisionTreeRegressor()

In [76]:
predict_value = model.predict(X)
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y, predict_value)

1115.7467183128902

- spilt test and train data

In [77]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)
model = DecisionTreeRegressor()
model.fit(train_X, train_y)
predict_value = model.predict(test_X)
mean_absolute_error(test_y, predict_value)

271823.50570260384

- control overfitting&underfitting by specify leaf nodes

In [78]:
def get_mae(max_leaf_nodes,train_X,test_X,train_y,test_y):
    my_model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
    my_model.fit(train_X, train_y)
    p_val = my_model.predict(test_X)
    mae = mean_absolute_error(test_y,p_val)
    return mae

- iteratively optimize model

In [79]:
import sys

nodes_num = 2 # initial value
step = 8
current_mae = sys.maxsize
while True:
    mae = get_mae(nodes_num,train_X,test_X,train_y,test_y)
    if current_mae-mae<1:
        break
    current_mae = mae
    nodes_num += step
print('current mae: {}, current leaf num: {}'.format(current_mae,nodes_num))

current mae: 269525.6425124663, current leaf num: 106


- use random forest instead of a single decision tree

In [82]:
from sklearn.ensemble import RandomForestRegressor

nodes_num = 2 # initial value
step = 10
current_mae = sys.maxsize
while True:
    forest_model = RandomForestRegressor(max_leaf_nodes=nodes_num)
    forest_model.fit(train_X,train_y)
    p_val = forest_model.predict(test_X)
    mae = mean_absolute_error(test_y,p_val)
    if current_mae-mae<1:
        break
    current_mae = mae
    nodes_num += step
print('mae: {}, leaf num: {}'.format(current_mae,nodes_num))

mae: 227681.39769992288, leaf num: 142
